In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from tqdm import tqdm
from pathlib import Path
import kagglehub
import shutil

/mnt/41A664F31125B500/Personal/Academics/4th_Year/1st_Sem/CSC173_Intelligent_Systems/CSC173-DeepCV-Bautista/venv/lib64/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Check CUDA availability

if cuda_available:= torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

device = 'cuda' if cuda_available else 'cpu'
print(f"CUDA Available: {cuda_available}")

GPU: NVIDIA GeForce RTX 3060 Laptop GPU
CUDA Available: True


In [3]:
kaggle_dataset_path = kagglehub.dataset_download("mexwell/silhouettes-for-human-posture-recognition")
CLASSES = ['sitting', 'standing']
src = Path(kaggle_dataset_path)
dst = Path("../../datasets/posture_classification")

dst.mkdir(parents=True, exist_ok=True)  # ensure target exists
for cls in CLASSES:
    src_cls = src / cls
    dst_cls = dst / cls
    if src_cls.exists():
        shutil.move(str(src_cls), str(dst_cls))

100%|██████████| 29.2M/29.2M [00:02<00:00, 12.8MB/s]

Extracting files...


In [4]:
# Paths
DATASET_DIR = str(dst)

# Classes

NUM_CLASSES = len(CLASSES)

# Training params
BATCH_SIZE = 32
EPOCHS = 50
LR = 1e-4
IMG_SIZE = 224


In [5]:
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [6]:
full_dataset = datasets.ImageFolder(
    root=DATASET_DIR,
    transform=train_transforms
)

# Train / Val split (80/20)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))

Train samples: 1920
Val samples: 480


In [7]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
DEVICE = torch.device(device)
# Freeze backbone (optional – comment out if full fine-tuning)
for param in model.parameters():
    param.requires_grad = False

# Replace classifier head
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

model = model.to(DEVICE)

In [8]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(
    model.fc.parameters(),   # train only classifier head
    lr=LR
)

In [9]:
def train_one_epoch(model, loader):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for imgs, labels in tqdm(loader):
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / len(loader), correct / total


In [10]:
def validate(model, loader):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / len(loader), correct / total


In [11]:
import time

def get_gpu_mem():
    if torch.cuda.is_available():
        return f"{torch.cuda.memory_allocated() / 1024**3:.2f}G"
    return "CPU"

for epoch in range(EPOCHS):
    model.train()
    start = time.time()

    running_loss = 0.0
    correct = 0
    total = 0

    print(
        f"Epoch {epoch+1}/{EPOCHS} "
        f"Size {IMG_SIZE} "
    )
    pbar = tqdm(train_loader, leave=False)
    for imgs, labels in pbar:
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total

    val_loss, val_acc = validate(model, val_loader)

    elapsed = time.time() - start

    print(
        f"GPU_mem {get_gpu_mem():>6} "
        f"loss {train_loss:.4f} "
        f"acc {train_acc:.3f} "
        f"val_loss {val_loss:.4f} "
        f"val_acc {val_acc:.3f} "
        f"time {elapsed//60:.0f}:{elapsed%60:04.1f}"
    )


Epoch 1/50 Size 224 


GPU_mem  0.08G loss 0.6231 acc 0.657 val_loss 0.5803 val_acc 0.713 time 0:10.1
Epoch 2/50 Size 224 


GPU_mem  0.08G loss 0.5447 acc 0.761 val_loss 0.5204 val_acc 0.767 time 0:09.6
Epoch 3/50 Size 224 


GPU_mem  0.08G loss 0.4913 acc 0.798 val_loss 0.4767 val_acc 0.817 time 0:10.1
Epoch 4/50 Size 224 


GPU_mem  0.08G loss 0.4543 acc 0.823 val_loss 0.4454 val_acc 0.827 time 0:09.8
Epoch 5/50 Size 224 


...

GPU_mem  0.08G loss 0.2351 acc 0.914 val_loss 0.2399 val_acc 0.906 time 0:10.8
Epoch 47/50 Size 224 


GPU_mem  0.08G loss 0.2382 acc 0.906 val_loss 0.2441 val_acc 0.896 time 0:10.7
Epoch 48/50 Size 224 


GPU_mem  0.08G loss 0.2273 acc 0.922 val_loss 0.2403 val_acc 0.900 time 0:10.5
Epoch 49/50 Size 224 


GPU_mem  0.08G loss 0.2280 acc 0.912 val_loss 0.2405 val_acc 0.902 time 0:10.9
Epoch 50/50 Size 224 


GPU_mem  0.08G loss 0.2232 acc 0.917 val_loss 0.2410 val_acc 0.902 time 0:10.7


In [13]:
torch.save(model.state_dict(), "../../runs/posture_classification/resnet18_posture_classification.pth")
print("Model saved!")

# ============================
# Stage 2: Full Fine-Tuning
# ============================
model.load_state_dict(
    torch.load("../../runs/posture_classification/resnet18_posture_classification.pth",
               map_location=DEVICE)
)

EPOCHS_STAGE2 = 8
FINE_TUNE_LR = 1e-5

# Unfreeze entire backbone
for param in model.parameters():
    param.requires_grad = True

# New optimizer for all parameters
optimizer = optim.Adam(model.parameters(), lr=FINE_TUNE_LR)

print("Backbone unfrozen. Starting full fine-tuning...")

for epoch in range(EPOCHS_STAGE2):
    model.train()
    start = time.time()

    running_loss = 0.0
    correct = 0
    total = 0

    print(
        f"Epoch {epoch+1}/{EPOCHS_STAGE2} "
        f"Size {IMG_SIZE} "
    )
    pbar = tqdm(train_loader, leave=False)
    for imgs, labels in pbar:
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total

    val_loss, val_acc = validate(model, val_loader)

    elapsed = time.time() - start

    print(
        f"GPU_mem {get_gpu_mem():>6} "
        f"loss {train_loss:.4f} "
        f"acc {train_acc:.3f} "
        f"val_loss {val_loss:.4f} "
        f"val_acc {val_acc:.3f} "
        f"time {elapsed//60:.0f}:{elapsed%60:04.1f}"
    )


# Save fine-tuned model
torch.save(
    model.state_dict(),
    "../../runs/posture_classification/resnet18_posture_classification_finetuned.pth"
)

print("Stage 2 fine-tuned model saved!")


Model saved!
Backbone unfrozen. Starting full fine-tuning...
Epoch 1/8 Size 224 


GPU_mem  0.20G loss 0.1585 acc 0.943 val_loss 0.1289 val_acc 0.958 time 0:12.7
Epoch 2/8 Size 224 


GPU_mem  0.20G loss 0.0790 acc 0.979 val_loss 0.1244 val_acc 0.965 time 0:12.6
Epoch 3/8 Size 224 


GPU_mem  0.20G loss 0.0483 acc 0.988 val_loss 0.1249 val_acc 0.958 time 0:13.2
Epoch 4/8 Size 224 


GPU_mem  0.20G loss 0.0288 acc 0.994 val_loss 0.1183 val_acc 0.963 time 0:13.3
Epoch 5/8 Size 224 


GPU_mem  0.20G loss 0.0173 acc 0.998 val_loss 0.1236 val_acc 0.963 time 0:13.3
Epoch 6/8 Size 224 


GPU_mem  0.20G loss 0.0106 acc 1.000 val_loss 0.1279 val_acc 0.958 time 0:13.8
Epoch 7/8 Size 224 


GPU_mem  0.20G loss 0.0107 acc 1.000 val_loss 0.1214 val_acc 0.963 time 0:14.5
Epoch 8/8 Size 224 


GPU_mem  0.20G loss 0.0079 acc 1.000 val_loss 0.1305 val_acc 0.963 time 0:14.9
Stage 2 fine-tuned model saved!
